In [ ]:
# conda installs
#! conda install elasticsearch

In [1]:
# importing necessary packages
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

import json
import gc

from elasticsearch import Elasticsearch


In [9]:
# creating elasticsearch client
elastic_client = Elasticsearch('http://192.168.100.146:9200', timeout = 30, max_retries = 10, retry_on_timeout = True)


In [4]:
# constructing DSL query
json_body = {'query':{
                'query_string':{
                                'query':'(host:4) and (host:12) and (host:15)'
                                }
                }
       }

# Lucene type query
lucene_q='(host:4) and (host:12) and (host:15)'


In [11]:
# trying to get count of records returned from Lucene query format
rec_count2 = elastic_client.count(q=lucene_q, index='batch3-*')

# observing the actual output from client.count
rec_count2


{'count': 3984430,
 '_shards': {'total': 160, 'successful': 160, 'skipped': 0, 'failed': 0}}

In [6]:
# trying to get count of records returned from DSL query format
rec_count = elastic_client.count(body=json_body, index='batch3-*')

# extracting only the 'count' attribute from the output
rec_count['count']


3984430

In [ ]:
# return more results using either scroll API or integer value on 'results'
# option, part of 'body' object
# half an hour to 45 minutes in this particula section for 3693170
'''
somecols = ['timestamp','statuscode','contentlength','host','timetoserv','hit','contenttype',
            'protocol','uid','sid','livechannel','contentpackage','assetnumber','coordinates',
            'uafamily']
'''

attributes = ['@timestamp','statuscode','contentlength','host','timetoserv','hit','contenttype',
              'protocol','uid','sid','livechannel','contentpackage','assetnumber','coordinates',
              'uafamily']

# executing search to extract scroll_id for the first num_recs
# records per page
num_recs = 10000
response = elastic_client.search(index='batch3-*',
#                                  _source=attributes,
                                 scroll='2m', 
                                 size=num_recs,
                                 #search_after,
                                 #body=json_body
                                 q=lucene_q
                                )

scroll_size = len(response['hits']['hits'])
print('initial scroll_size:', scroll_size)
# response['hits']['hits']
#getting the initial scroll_id from initial client.search results
sid = response['_scroll_id']
print('initial scroll_id')
# #print(response['hits']['hits'])

df = pd.DataFrame()
print('empty dataframe')
appended_data = []
print('appended_data to hold list of dataframes')
k = 0
while (scroll_size > 0):
    frame = pd.DataFrame.from_dict([document['_source'] for document in response['hits']['hits']])
    appended_data.append(frame)
    response = elastic_client.scroll(scroll_id = sid, scroll = '1m', request_timeout = 30)
    # Update the scroll ID
    print('updating scroll_id for page',k)
    sid = response['_scroll_id']
    # Get the number of results that we returned in the last scroll
    scroll_size = len(response['hits']['hits'])
    k += 1
if len(appended_data) > 0:
    df = pd.concat(appended_data, ignore_index = True, sort = False)
    print('concatenated all dataframes in the appended_data list')
del appended_data
gc.collect()
es.clear_scroll(body = {'scroll_id': sid})
print('finalizing the scroll_id')

# creating empty list to hold all elastic search responses
# elastic_response = []

# adding num_recs records of the first scroll_id
# elastic_response.append(response['hits']['hits'])

# counts=len(response['hits']['hits'])
# print('records returned from client.search:',counts)

# just in case, assigning the output to another variable
# this constitutes the first 10000 records based on client.search
# el_docs = response

# new list for Elasticsearch documents
#elastic_docs = response['hits']['hits']

# looping the remaining records in client.scroll.
# total_rec = 0
# while(len(response['hits']['hits']) != 0):
#     # remaining records of subsequent scroll_id in this section
#     response=elastic_client.scroll(scroll_id=old_scroll_id, scroll='2m')
    
#     #the next _scroll_id to iterate through
#     old_scroll_id=response['_scroll_id']
    
#     #adding num_recs records of the first scroll_id
#     elastic_response.append(response['hits']['hits'])
    
#     #print('this scroll id:',old_scroll_id,' \n ')
#     total_rec+=len(response['hits']['hits'])
#     #print('records returned from client.scroll:',total_rec)

# print('records returned from client.scroll:',total_rec)
# print('overall number of record pages in list:',len(elastic_response))

records returned from client.search: 10000


### parallel processing to get the dataset

In [ ]:
# function for multiprocessing scrolling API to get dataset from elasticsearch {original piece}
# def es_scroll(index, min_timestamp, max_timestamp, slice_no):
#     es = Elasticsearch('http://192.168.100.146:9200', timeout = 30, max_retries = 10, retry_on_timeout = True)
#     page = es.search(index = index,
#                      doc_type = 'tls_book',
#                      scroll = '1m',
#                      body = {
#                          'slice': {
#                              'id': slice_no,
#                              'max': SLICES
#                          },
#                          '_source': [
#                              'host', 'contentpackage'
#                          ],
#                          'sort': [
#                              '_doc'
#                          ],
#                          'query': {
#                              'range': {
#                                  '@timestamp': {
#                                      'gte': min_timestamp,
#                                      'lte': max_timestamp,
#                                      'boost': 2.0
#                                  }
#                              }
#                          }
#                      },
#                      version = False,
#                      size = 10000
#                     )
#     sid = page['_scroll_id']
#     scroll_size = page['hits']['hits']
    
#     # Start scrolling
#     df = pd.DataFrame()
#     appended_data = []
    
#     while (scroll_size > 0):
#         frame = pd.DataFrame.from_dict([document['_source'] for document in page ['hits']['hits']])
#         appended_data.append(frame)
#         page = es.scroll(scroll_id = sid, scroll = '1m', request_timeout = 30)
#         # Update the scroll ID
#         sid = page['_scroll_id']
#         # Get the number of results that we returned in the last scroll
#         scroll_size = len(page['hits']['hits'])
#     if len(append_data) > 0:
#         df = pd.concat(appended_data, ignore_index = True, sort = False)
#     del appended_data
#     gc.collect()
#     es.clear_scroll(body = {'scroll_id': sid})
#     return df
    
    
    

In [5]:
# function for multiprocessing scrolling API to get dataset from elasticsearch {modified piece}
'''
this code works wonderfully!
'''
def es_scroll(index):
    es = Elasticsearch('http://192.168.100.146:9200', timeout = 30, max_retries = 10, retry_on_timeout = True)
    page = es.search(index = index,
                     # doc_type = 'tls_book',
                     scroll = '2m',
                     # body = json_body,
                     q = lucene_q,
                     #version = False,
                     size = 10000
                    )
    sid = page['_scroll_id']
    # print('sid:', sid)
    scroll_size = len(page['hits']['hits'])
    
    print('records:', page['hits']['hits'])
    
    # Start scrolling
    df = pd.DataFrame()
    appended_data = []
    
    print('records returned from client.search:', scroll_size)
    
    while (scroll_size > 0):
        frame = pd.DataFrame.from_dict([document['_source'] for document in page['hits']['hits']])
        appended_data.append(frame)
        page = es.scroll(scroll_id = sid, scroll = '2m', request_timeout = 30)
        # Update the scroll ID
        sid = page['_scroll_id']
        # Get the number of results that we returned in the last scroll
        scroll_size = len(page['hits']['hits'])
        
    print('size of appended data:', len(appended_data))
    if len(appended_data) > 0:
        df = pd.concat(appended_data, ignore_index = True, sort = False)
    del appended_data
    gc.collect()
    es.clear_scroll(body = {'scroll_id': sid})
    return df

In [6]:
vod_df = es_scroll(index='batch3-*')

/opt/conda/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

/opt/conda/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/

/opt/conda/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
/opt/conda/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
/opt/conda/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessibl

/opt/conda/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
/opt/conda/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
/opt/conda/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessibl

/opt/conda/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
/opt/conda/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
/opt/conda/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessibl

/opt/conda/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
/opt/conda/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
/opt/conda/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessibl

...

/opt/conda/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
/opt/conda/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
/opt/conda/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessibl

/opt/conda/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
/opt/conda/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
/opt/conda/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessibl

/opt/conda/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
/opt/conda/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


size of appended data: 399


/tmp/ipykernel_115/1717982381.py:38: DeprecationWarning: The 'body' parameter is deprecated for the 'clear_scroll' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.clear_scroll(body = {'scroll_id': sid})


In [7]:
vod_df.shape

(3984430, 30)

In [9]:
vod_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3984430 entries, 0 to 3984429
Data columns (total 30 columns):
 #   Column          Dtype  
---  ------          -----  
 0   devicemodel     object 
 1   popname         object 
 2   uid             object 
 3   timetoserv      float64
 4   maxage          object 
 5   devicefamily    object 
 6   fragment        bool   
 7   coordinates     object 
 8   hit             object 
 9   uamajor         object 
 10  cachecontrol    object 
 11  timefirstbyte   float64
 12  livechannel     object 
 13  contentpackage  object 
 14  statuscode      int64  
 15  uafamily        object 
 16  osfamily        object 
 17  contentlength   float64
 18  @timestamp      object 
 19  @version        object 
 20  host            object 
 21  method          object 
 22  assetnumber     object 
 23  path            object 
 24  cachename       object 
 25  sid             object 
 26  protocol        object 
 27  manifest        bool   
 28  devicebrand 

In [10]:
vod_df.to_csv('CSV/VoD_ELK_Parallel.csv', index = False)

In [53]:
# # new list for Elasticsearch documents
# elastic_docs = response['hits']['hits']

# making a copy of elastic_response list
elastic_docs = elastic_response.copy()
print('overall number of records pages in list:',len(elastic_docs))

overall number of records pages in list: 400


In [ ]:
'''
Store Elasticsearch index's fields in a dictionary

Recommended to process the generated lists from elasticsearch scrolling in batches
and each batch separately append to dataframe.
Write the CSVs in chunks.

Also you can do multi-threading for the list to dictionary to dataframe then
combine the generated individual dataframes.

Hopefully will mitigate this bottleneck

'''
# empty dictionary for Elasticsearch fields
fields = {}

# # attempts at parallelizing the operation by two sub
# # dictionaries
# fields1 = {}
# fields2 = {}
# indexer for our list, might not be necessary
# 15 minutes per 10 pages

i = 0
pages = len(elastic_docs)

# bottleneck right in this loop
# iterate over the document list returned by API call
while(i < 50):
    for num, doc in enumerate(elastic_docs[i]):
        # get source data from document
        source_data = doc['_source']
        
        #try with a dictionary comprehension
#         try:
#             fields = {key:np.append(fields[key],val) for key, val in source_data.items()}
#         except KeyError:
#             fields[key] = np.array([val])
        
        # iterate source data
        for key, val in source_data.items():
            try:
                fields[key] = np.append(fields[key], val)
            except KeyError:
                fields[key] = np.array([val])
    print('page',i)
    i += 1
    

# iterate key-value pairs of the fields dictionary
# for key, val in fields.items():
#     #print(key, '', val)
#     print('Numpy array len:', len(val), '\n')

page 0
page 1
page 2
page 3
page 4
page 5
page 6
page 7
page 8
page 9
page 10
page 11
page 12
page 13
page 14
page 15
page 16
page 17
page 18
page 19
page 20
page 21


In [1]:
'''
Create a DataFrame object from elasticsearch
fields data
'''
# creating a pandas dataframe array from fields dictionary
elastic_df = DataFrame(fields)

# renaming one of the fields
#elastic_df.rename(columns={'@timestamp':'timestamp'}, inplace=True)

print('elastic_df:', type(elastic_df), '\n')
print(elastic_df.shape)
#print(elastic_df.info())
elastic_df.to_csv('CSV/cdnelk_VoD1.csv')
elastic_df.tail(6)

NameError: name 'DataFrame' is not defined

In [ ]:
elastic_df = elastic_df.fillna(value=np.nan)
print(elastic_df.info())
elastic_df

In [ ]:
print('elastic_df:', type(elastic_df), '\n')

# make columns to appropriate types
elastic_df['@timestamp'] = pd.to_datetime(elastic_df['@timestamp'])
elastic_df.astype({'livechannel': np.float64, 'contentpackage': np.float64})

print(elastic_df.info())
elastic_df

In [ ]:
# wondering why it refuses to change the dtypes to float???
elastic_df['livechannel'].astype(float)
elastic_df['contentpackage'].astype(float)
print(elastic_df['livechannel'].dtypes)
print(elastic_df['contentpackage'].dtypes)

In [ ]:
# create a JSON string from Pandas object
json_data = elastic_df.to_json()
# print('\nto_json() method:', json_data)

In [ ]:
# verify that the to_jason() method made a JSON string
try:
    json.loads(json_data)
    print('\njson_data is a valid JSON string')
except json.decoder.JSONDecodeError as err:
    print('\njson.decoder.JSONDecodeError:', err)
    print('json_data is NOT a valid JSON string')



In [ ]:
'''
Creating Series object from Elasticsearch documents
'''

# # an empty dictionary for series arrays
# elastic_series = {}
# #initializing new indexer for the list of elastic_docs
# j = 0

# # iterate the docs returned from API call
# while(j < len(elastic_docs)):
#     for num, doc in enumerate(elastic_docs[j]):

#         # get the _id for the doc
#         _id = doc['_id']

#         # get the source data from document
#         source_data = doc['_source']

#         # make a Series object for the doc using _id as key
#         elastic_series[_id] = Series()

#         # iterate source_data
#         for field, value in source_data.items():

#             # set the field type as Series index and value as Series value
#             elastic_series[_id].at[field] = value
#     j += 1
        
# for key, doc in elastic_series.items():
#     print('\nID:', key, '\n', doc)